In [1]:
import trimesh
import numpy as np
from collections import Counter
import time
import math
from tqdm import tqdm
import os
from pathlib import Path
import pymeshfix

In [4]:
import math
def area(vertices):
    """
    Calculates the area of a 3D triangle from it's coordinates
    """
    side_a = np.linalg.norm(vertices[0]-vertices[1])
    side_b = np.linalg.norm(vertices[1]-vertices[2])
    side_c = np.linalg.norm(vertices[2]-vertices[0])
    s = 0.5 * ( side_a + side_b + side_c)
    return math.sqrt(s * (s - side_a) * (s - side_b) * (s - side_c))

def find_polygon_area(mesh,list_of_faces):
    "Calculates the area of a 3D polygon that is created from connected traingles"
    return(sum([area(mesh.vertices[mesh.faces[r]]) for r in list_of_faces]))


#filters by convexity and generates the facet centers

def filter_final_facets_working(gap_mesh,final_facets,adjacency_threshold =  0.8):
    """
    Gets the facets faces list and the center points of these facets from mesh
    Filters:
    1) Only lets facets greater than first_pass_size_threshold exist
      **** might need to look at this because area is that of facets before expansion
    2) Has to have a high convex border
    
    Expansions:
    1) Expands the facet group to neighbors that are within the normal_closeness
    for their normals when doing the dot product
    
    Order:
    1) Size filtering
    2) Expansion
    3) Convex Border filtering
    """


    # after computing final faces, filter for convexity
    edges = gap_mesh.edges_sorted.reshape((-1, 6)) #groups all the edges belonging to the corresponding face in one row
    final_facets_mean = np.zeros(len(final_facets))
    
    
    #make lookup table for face number to spot in the adjacency edges
    face_adjacency_index_lookup = [[] for i in gap_mesh.faces]
    for i,faces in enumerate(gap_mesh.face_adjacency):
        for f in faces:
            face_adjacency_index_lookup[f].append(i)


    for j,facet in enumerate(final_facets):
        # get the edges for each facet
        edges_facet = [edges[i].reshape((-1, 2)) for i in [facet]][0] #stores all the edges belonging to that face

        #get the indexes of the boundary edges:
        indexes = trimesh.grouping.group_rows(edges_facet, require_count=1)
        edge_0 = edges_facet[indexes]

        #find the faces that correspond to the boundary edges
        edge_0_faces = [facet[int(k/3)] for k in indexes]


        #2) Find the indexes of the edges int he face_adajacency_edges and store the projections
        adjacency_values = []
        for edge,edge_face in zip(edge_0,edge_0_faces):
            possible_adj_indexes = face_adjacency_index_lookup[edge_face]

            for index in possible_adj_indexes:
                if len(set(edge).intersection(set(gap_mesh.face_adjacency_edges[index]))) >= 2:
                    #print(f"adj edge = {e} and boundary edge = {edge}")
                    adjacency_values.append(gap_mesh.face_adjacency_angles[index]) # the metric we actually want to measure
                    break


        final_facets_mean[j] = np.mean(adjacency_values)
        
        

    #filter the final facets and output them so they can be plotted
    
    
#     #need to make sure they are array of arrays
#     thresholld_boolean_results = final_facets_mean > adjacency_threshold
#     for fa,individual_facet in  enumerate(final_facets):
#         if thresholld_boolean_results[fa] == True:
#             np
    
    final_facets_mean_filtered = np.array(final_facets)[final_facets_mean > adjacency_threshold]
    
    if len(final_facets_mean_filtered.shape) > 1:
        total_array = np.empty(final_facets_mean_filtered.shape[0],object)
        total_array[:] = [x for x in final_facets_mean_filtered]
        final_facets_mean_filtered = total_array
        print("Had to restructure the array because was 2D array")

    

    #Compute the centers
    final_facets_centers = []
    
    for filt in final_facets_mean_filtered: 
        #print("filt = " + str(filt))
        unique_vertices = gap_mesh.vertices[np.unique(gap_mesh.faces[filt].ravel())].astype("float")
        final_facets_centers.append((np.mean(unique_vertices[:,0]),
                          np.mean(unique_vertices[:,1]),
                          np.mean(unique_vertices[:,2])))
    
    return final_facets_mean_filtered,final_facets_centers


#main function that generates the facets
def filter_final_facets_optimized_with_checks(example_mesh,min_len=2,
                                  normal_closeness=0.99,
                                  first_pass_size_threshold=6000,
                                  adjacency_threshold =  0.8
                                  ):
    """
    Way of computing facets that uses trimesh grouping function and normals threshold
    instead of the pre-built trimesh.facets
    
    """

    #get the facets for the child mesh
    start_time_total = time.time()

    #switch out with dot product
    
    
    #Old way of computing the dot product manually
    total_normals = example_mesh.face_normals[example_mesh.face_adjacency]
    
    #using manual method
#     print(len(total_normals[:,0]*total_normals[:,1]))
    start_normal = time.time()
    total_normal_dots = np.sum(total_normals[:,0]*total_normals[:,1],axis=1)
    #print(f"Total normals took {time.time() - start_normal}")
    ''' DONT HAVE TO DO DOT PRODUCT BECAUSE JUST WANT TO MULTIPLY THE ROWS OF EACH COLUMN'''
#     print("About to do dot product")
#     dot_start = time.time()
#     a = total_normals[:,0]
#     b = total_normals[:,1].T
#     #new way using the dot product
#     total_normal_dots = np.dot(a,b)
    
    
    #get the True/False value if face adjacency is within normal closeness
    start_normal = time.time()
    total_normal_dots_facet_mask = total_normal_dots > normal_closeness
    #print(f"Boolean mask finished: {time.time() - start_normal}")

    #getting the grouping of the faces into facets within the threshold
    start_normal = time.time()
    components = trimesh.graph.connected_components(example_mesh.face_adjacency[total_normal_dots_facet_mask],
                                          nodes=np.arange(len(example_mesh.faces)),
                                          min_len=min_len,
                                          engine="None")
    #print(f"Grouping took: {time.time() - start_normal}")
    
    

    #print(f"Lenght of facets BEFORE filtering = {len(components)}")
    
    #filter by the size
    
    start_normal = time.time()
    size_filtered_components = [facet_list for facet_list in components 
                                    if find_polygon_area(example_mesh,facet_list) > first_pass_size_threshold]
    print(f"Filtering edges by size finished: {time.time() - start_normal}, facet # = {len(size_filtered_components)}")

#     if facet_index in checking_list:
#         print("size_filtered_components = " + str(size_filtered_components))

    #filter by the convexity
    #final_facets_mean_filtered,final_facets_centers = filter_final_facets(example_mesh,size_filtered_components)
    
    start_normal = time.time()
    final_facets_mean_filtered,final_facets_centers = filter_final_facets_working(example_mesh,size_filtered_components,adjacency_threshold)
    print(f"Filtering by convexity and getting centers took: {time.time() - start_normal}, facet # = {len(final_facets_mean_filtered)}")

#     if facet_index in checking_list:
#         print("final_facets_mean_filtered = " + str(final_facets_mean_filtered))
    
    #print(f"Total Time = {time.time() - start_time_total}")

    return final_facets_mean_filtered,final_facets_centers
    

In [13]:
#get the facets for the main mesh
main_mesh = trimesh.load_mesh("./neurons/648518346349482676_stitch_test.off")
main_mesh.show()

/usr/local/lib/python3.6/dist-packages/IPython/core/display.py:689: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")


In [14]:

min_len_large=3
normal_closeness_facets=0.99
first_pass_size_threshold = 6000
adjacency_threshold = 0.8
# find the facets on the main mesh 
main_mesh_facets,main_mesh_facets_centers = filter_final_facets_optimized_with_checks(example_mesh=main_mesh,
                                                                                          min_len=min_len_large,
                                                                                              normal_closeness=normal_closeness_facets,
                                                                                              first_pass_size_threshold=first_pass_size_threshold,
                                                                                              adjacency_threshold=adjacency_threshold)
        

Filtering edges by size finished: 0.01060795783996582, facet # = 29
Filtering by convexity and getting centers took: 0.024845361709594727, facet # = 10


In [16]:
segment_id = 648518346349482676

main_mesh_facet_face = 420
child_facet_face = 258

In [ ]:
# download the facets and make sure they match

current_facets = main_mesh_facets
current_mesh = main_mesh
index = "main"


facets_group = np.zeros(len(current_mesh.faces)).astype(int)

for i,facet_group in enumerate(current_facets):
    for face in facet_group:
        facets_group[face] = i + 1 #so that you reserve the label 0 for blenders none

np.savez("./test_meshes/" + str(segment_id) + "_debug_stitching_" + str(index) + ".npz",
         facets_group=facets_group)

# from print_trimesh import print_trimesh
# print_trimesh(current_mesh,"./test_meshes/piece_debug_june_" + str(index) + ".off")




In [17]:
for i,facet in enumerate(main_mesh_facets):
    if child_facet_face in facet:
        print(f"Child = {i,facet}")
    if main_mesh_facet_face in facet:
        print(f"Main = {i,facet}")

Main = (1, array([637,  60, 635,  59, 420, 628, 600, 421]))
Child = (4, array([206, 341, 530, 577, 342, 578, 356, 254, 316, 397, 258, 257, 579,
       563]))


# Do the stitching

In [ ]:
main_mesh
facet_1 = [637,  60, 635,  59, 420, 628, 600, 421]
facet_2 = [206, 341, 530, 577, 342, 578, 356, 254, 316, 397, 258, 257, 579, 563]

In [96]:

def apply_bbox_filter(child,min_bb_zone,max_bb_zone):
    """
    Determines if child is withing the bounding box zone
    designated by the bounding box corners
    
    """
    #get the min and max of the bounding box for the mesh
    min_bb = np.array(child.bounding_box.vertices).min(0)
    max_bb = np.array(child.bounding_box.vertices).max(0)
    
    #print(min_bb,max_bb)
    #print(min_bb_zone,max_bb_zone)
    
    #if fails any of these checks then return false, else return True
    if min(min_bb[0],max_bb[0])>max_bb_zone[0]:
        print("returning x greater max")
        return False
    
    if max(min_bb[0],max_bb[0])<min_bb_zone[0]:
        print("returning x less min")
        return False
    
    if min(min_bb[1],max_bb[1])>max_bb_zone[1]:
        print("returning y greater max")
        return False
    
    if max(min_bb[1],max_bb[1])<min_bb_zone[1]:
        print("returning y less min")
        return False
        
    if min(min_bb[2],max_bb[2])>max_bb_zone[2]:
        print("returning z greater max")
        return False
    
    if max(min_bb[2],max_bb[2])<min_bb_zone[2]:
        print("returning z less mim")
        return False
    return True

import math
def area(vertices):
    """
    Calculates the area of a 3D triangle from it's coordinates
    """
    side_a = np.linalg.norm(vertices[0]-vertices[1])
    side_b = np.linalg.norm(vertices[1]-vertices[2])
    side_c = np.linalg.norm(vertices[2]-vertices[0])
    s = 0.5 * ( side_a + side_b + side_c)
    return math.sqrt(s * (s - side_a) * (s - side_b) * (s - side_c))

def find_polygon_area(mesh,list_of_faces):
    "Calculates the area of a 3D polygon that is created from connected traingles"
    return(sum([area(mesh.vertices[mesh.faces[r]]) for r in list_of_faces]))

# restitching functions
import trimesh
import numpy as np
from collections import Counter
import time
import math

#gets the projection of point p onto line a
def ClosestPointOnLine(a, b, p):
    ap = p-a
    ab = b-a
    #base_vector = ab
    result = np.dot(ap,ab)/np.dot(ab,ab) # * ab
    return result



#now have the mesh and the facet faces, can send to function
def stitch_mesh_piece_vp4(new_mesh,facet_1,facet_2,
                          delete_facets=False,
                         return_added_mesh = False,
                         fix_normals = False):

    """
    Changed since last version: 
    1) parameter for deleting facets at end or not
    2) parameter for returning added mesh or not 
    3) Changed normals check to print statement and not exception


    """
    print("statistcs on the entire mesh")
    print("len(new_mesh.faces) = " + str(len(new_mesh.faces)))
    print("len(new_mesh.vertices) = " + str(len(new_mesh.vertices)))
    
    print("facet_1 = " + str(facet_1))
    print("facet_2 = " + str(facet_2))
    
    #how to find the normals of facet groups:
    facet_group_1_normal = new_mesh.face_normals[facet_1[0]] #main_mesh_normals
    facet_group_2_normal = new_mesh.face_normals[facet_2[0]] #child_mesh_normals
    
    print(f"Main normal = {facet_group_1_normal}")
    print(f"Child normal = {facet_group_2_normal}")


    #get the correct version of the normals: (might need to flip them if going in opposite direction)
    if np.dot(facet_group_1_normal,facet_group_2_normal) > 0.8:
        raise Exception("same direction normals")
    elif np.dot(facet_group_1_normal,facet_group_2_normal) < -0.8:
        print("opposite direction normals")
    else:
        print("Not correct normals")
        #raise Exception("Not correct normals")

    # make each row correspond to a single face 
    edges = new_mesh.edges_sorted.reshape((-1, 6))
    # get the edges for each facet
    edges_facet = [edges[i].reshape((-1, 2)) for i in [facet_1,facet_2]]
    print("edges_facet = " + str(edges_facet))
    edges_boundary = np.array([i[trimesh.grouping.group_rows(i, require_count=1)]
                               for i in edges_facet])
    
    print("edges_boundary = " + str(edges_boundary))

    #the list of boundary edges and unique points in the boundary edges
    edge_0 = edges_boundary[0]
    edge_1 = edges_boundary[1]

    #gets the unique number of points
    edge_0_points = np.unique(np.hstack(edge_0))
    edge_1_points = np.unique(np.hstack(edge_1))
    print("Found boundary edges")
    """
    get the dot product of all the points
    """

    print(f"edge_0_points = {edge_0_points}")
    print(f"edge_1_points = {edge_1_points}")

    #get any 2 points on the triangle and make that the reference edge
    print("edge_point_0 = " + str(edge_0_points[0]))
    print("edge_point_1 = " + str(edge_0_points[1]))
    edge_0_anchor_points = new_mesh.vertices[[edge_0_points[0],edge_0_points[1]]]
    
    print(edge_0_anchor_points)

    #gets the starting index for the 1st facet (so that the start of the stitching is close)
    max_index = 0
    max_magnitude = ClosestPointOnLine(edge_0_anchor_points[0],edge_0_anchor_points[1],new_mesh.vertices[edge_0_points[max_index]])

    print("Starting max magnitude 0" + str(max_magnitude))
    
    for i in range(1,len(edge_0_points)):
        print(f"0: current vertex {edge_0_points[i]} with coordinate {new_mesh.vertices[edge_0_points[i]]}")
        current_magnitude = ClosestPointOnLine(edge_0_anchor_points[0],edge_0_anchor_points[1],new_mesh.vertices[edge_0_points[i]])
        print("current_magnitude = " + str(current_magnitude))
        if current_magnitude > max_magnitude:
            print("changing the max magnitude")
            max_index = i
            max_magnitude = current_magnitude

    edge_0_starting_point = edge_0_points[max_index]


    #gets the starting index for the 2nd facet (so that the start of the stitching is close)
    max_index = 0
    max_magnitude = ClosestPointOnLine(edge_0_anchor_points[0],edge_0_anchor_points[1],new_mesh.vertices[edge_1_points[max_index]])

    print("Starting max magnitude for 1" + str(max_magnitude))
    
    for i in range(1,len(edge_1_points)):
        print(f"1: current vertex {edge_1_points[i]} with coordinate {new_mesh.vertices[edge_1_points[i]]}")
        current_magnitude = ClosestPointOnLine(edge_0_anchor_points[0],edge_0_anchor_points[1],new_mesh.vertices[edge_1_points[i]])
        print("current_magnitude = " + str(current_magnitude))
        if current_magnitude > max_magnitude:
            print("changing the max magnitude")
            max_index = i
            max_magnitude = current_magnitude

    edge_1_starting_point = edge_1_points[max_index]

    print(f"starting edge 1st facet = {edge_0_starting_point}, starting edge 2nd facet= {edge_1_starting_point}, ")
    #print(new_mesh.vertices[edge_0_starting_point],new_mesh.vertices[edge_1_starting_point])

    """
    Need to order the points for restitching

    Pseudocode: 
    1) Get starting piont
    2) Find the two edges corresponding to that point
    3) Need to decide which direction to start....
    - go in direction that make the cross of the (1st and last) point in the same direction of the 
    normal of the first facet
    4) loop through and record the orders of the vertices as you traverse along the edges 
    until you arrive back at the start
    5) Error if:
        a. You arrive back at the start and haven't processed all the edges
        b. Processsed all the edges but haven't arrived back at start

    6) Repeat steps 1 through 5 for 2nd facet group
    """
    start_point_list = [edge_0_starting_point,edge_1_starting_point]
    print("start_point_list = " + str(start_point_list))
    edge_list = [edge_0,edge_1]
    edge_order_list = []



    #loop that organizes the unique boundary points into the correct order
    for i,start_point in enumerate(start_point_list):
        print(f"Starting Organizing vertices for side {i}")
        #print(f"start_point = {start_point}")
        edge_order = [start_point]
        processed_edges = []

        #find the matching edges to the starting point
        starting_edges_indices = np.where(np.logical_or(edge_list[i][:,0] == start_point,edge_list[i][:,1] == start_point) == True)[0]

        starting_edges = edge_list[i][starting_edges_indices]
        #print(f"starting edges = {starting_edges}") #the list of the two possible edges

        if starting_edges.size < 4:
            raise Exception("Not enough edges for 1st facet start point")

        if starting_edges.size > 4:
            raise Exception("Too many edges for 1st facet start point") 

        #np.where(starting_edges[1,:] != start_point)[0][0]
        #gets the vectors that will be used for the cross product
        #print("np.where(starting_edges[0,:] != start_point)[0][0] = " + str(np.where(starting_edges[0,:] != start_point)[0][0]))
        #print("np.where(starting_edges[1,:] != start_point)[0][0] = " + str(np.where(starting_edges[1,:] != start_point)[0][0]))

        """*************** where pick the starting edge starts ************
        The way it works: 
        1) Gets the two possible starting edges
        2) Generates the vectors for the edges where origin is the starting point
        3) Gets the cross porduct of both vectors
        4) Chooses the cross product that is in the direction of the face normals

        Why that doesn't work:
        1) they are opposite normals


        """

        """
        Possible other solution:
        1) Get the starting points on the child edge
        2) Pick the first edge as the default edge

        """
        processed_edges.append(starting_edges_indices[0])
        current_vertex = starting_edges[0][np.where(starting_edges[0,:] != start_point)[0][0]]




    #         #gets the possible starting vectors from the two possible edges
    #         possible_starting_vector_1 = new_mesh.vertices[starting_edges[0,:][np.where(starting_edges[0,:] != start_point)[0][0]]] - new_mesh.vertices[start_point]
    #         #just start with a random edge
    #         possible_starting_vector_2 = new_mesh.vertices[starting_edges[1,:][np.where(starting_edges[1,:] != start_point)[0][0]]] - new_mesh.vertices[start_point]


    #         #find the cross product of the starting vectors
    #         starting_edges_cross = np.cross(possible_starting_vector_1,possible_starting_vector_2)

    #         #make sure that the order of the vectors goes so that the cross product is in line with the starting normal
    #         #this ensures the the circular direction of the stitching will be the same
    #         if np.dot(starting_edges_cross,facet_group_1_normal) > 0:
    #             print("Edge 1 picked for direction")
    #             processed_edges.append(starting_edges_indices[0])
    #             current_vertex = starting_edges[0][np.where(starting_edges[0,:] != start_point)[0][0]]
    #         else:
    #             print("Edge 2 picked for direction")
    #             processed_edges.append(starting_edges_indices[1])
    #             #print("np.where(starting_edges[1,:] != start_point) = " + str(np.where(starting_edges[1,:] != start_point)))
    #             current_vertex = starting_edges[1][np.where(starting_edges[1,:] != start_point)[0][0]]

        #print(f"current_vertex = {current_vertex}" )
        #print("edge_list = " + str(edge_list))



        """*************** where pick the starting edge ends ************"""
        #now iterate through number of 
        for z in range(1,edge_list[i][:,0].size):
            #print("edge_order_temp = " + str(edge_order))
            if current_vertex == start_point:
                print("Start vertex reached before processed all of edges")

                """

                These should be ok because the extra loops are created from holes inside and this process should always get the outside loop


                """
                break

            #get the next edge
            counter = 0
            next_vertex = -1
            for j,edg in enumerate(edge_list[i]):
                #print("edg = " + str(edg))
                #print("processed_edges = " + str(processed_edges))
                if current_vertex in edg and j not in processed_edges:
                    current_edge_index = j
                    if edg[0] != current_vertex:
                        next_vertex = edg[0]
                    else:
                        next_vertex = edg[1]


                    counter += 1
                    if counter >= 2:
                        #raise Exception(f"More than 2 edges possibilities for {current_vertex}")
                        #Don't want to make it an exception anymore put just print out warning
                        print("More than 2 edges possibilities for {current_vertex}") # BAC change

            #make sure the next vertex was found
            if next_vertex <= -1:
                raise Exception(f"No next vertex was found for {current_vertex} ")

            #if found next vertex then add the old vertex and edge index
            #to the processed edges lists and the order of vertices
            processed_edges.append(current_edge_index)
            edge_order.append(current_vertex)

            current_vertex = next_vertex


        edge_order_list.append(edge_order)
        print(f"edge_{i}_order done, len = {len(edge_order)} ")#"= {edge_order}")

        #print the edge orders
        for e in edge_order_list:
            print(e)
    lengths_of_boundaries = [len(x) for x in edge_order_list]



    """ ************ PROCESS OF ORDERING THE EDGE PATHS SO THAT *********
    main mesh loop goes in counter clockwise in reference to the gap
    child goes clockwise in reference to gap




    """

    """
    1) Pick the starting point as your P point
    2) For each point in ordered list calculate point - P and store that vector
    3) Do the cross product of list[0:n-2] x list[1:n-1]
    4) Take the sum of these lists of cross products
    5) Do the dot product to compare the direction with the normal vector 
    Result: 
    - if positive --> conuter-clockwise according to normal
    - if negative --> clockwise according to normal

    """

    starter_point = np.array([0,0,0])

    #     list_of_points = [1,2,3,4,0]

    #     list_of_points = [list_of_points[len(list_of_points) - x -1] for x in range(0,len(list_of_points))]
    #     print(list_of_points)

    #print("facet_group_2_normal = " + str(facet_group_2_normal))

    for ed,e_loop in enumerate(edge_order_list):

        #get the vertices according to the points
        vertices_list = new_mesh.vertices[e_loop]

        #get the cross product of the offsetted list
        #vertices_list[0:len(vertices_list)-1,:]
        """ Wrong earlier previous way

        cross_products = np.cross(vertices_list[0:len(vertices_list)-1,:],vertices_list[1:len(vertices_list)])
        """

        cross_products = np.cross(vertices_list[0:len(vertices_list),:],
                             np.vstack([vertices_list[1:len(vertices_list),:],vertices_list[0,:]]))

        sum_cross_products = np.sum(cross_products,axis=0)

        #print("cross_products = " + str(cross_products))
        #print("sum_cross_products = " + str(sum_cross_products))

        #print("Before edge list = " + str(edge_order_list[ed]))
        if ed == 0:
            #get the dot product
            normals_dot = np.dot(sum_cross_products,facet_group_1_normal)
            #print(' normals_dot = ' + str(normals_dot))
            if normals_dot > 0:
                print("Main originally was counter-clockwise --> keeping")


            else:
                print("Main originally was clockwise --> flipping")
                new_loop = [e_loop[len(e_loop) - x -1] for x in range(0,len(e_loop)-1)]
                new_loop.insert(0,e_loop[0])
                                                                                                          
                                                                                                          
                edge_order_list[ed] = new_loop.copy()

        else: 
            # for the children want the cross product to be counter clockwise
            normals_dot = np.dot(sum_cross_products,facet_group_2_normal)
            #print(' normals_dot = ' + str(normals_dot))
            if normals_dot > 0:
                print("Child originally was counter-clockwise --> flipping")
                new_loop = [e_loop[len(e_loop) - x -1] for x in range(0,len(e_loop)-1)]
                new_loop.insert(0,e_loop[0])
                                                                                                          
                                                                                                          
                edge_order_list[ed] = new_loop.copy()
            else:
                print("Child originally was clockwise --> keeping")


        #print("After edge list = " + str(edge_order_list[ed]))
    """  SHOWS THAT DOING THE CROSS PRODUCT THAT WAY WORKS
    #do the cross products manually
    for jj in range(0,len(vertices_list)-1):
        print(np.cross(vertices_list[jj],vertices_list[jj+1]))

    """

    #print("edge_order_list = " + str(edge_order_list))



    #getting which one is the bigger one
    bigger = lengths_of_boundaries.index(max(lengths_of_boundaries))
    smaller = 1-bigger

    if bigger == 0:
        starter_face = "child" #if the bigger one was the main, then child is the smaller one you start from
    else:
        starter_face = "main" #if the bigger one was the child, then main is the smaller one you start from


    print("smaller_face = " + str(starter_face))    

    """ The rules that have to be following in order for normals to be correctly aligned
    1) if the smaller is the child (will be traveling in clockwise direction
    --> need to stitch points as:
    Regular: other_2, other_1,current_point
    Neighbor: current,other_1,current-1


    1) if the smaller is the main mesh (
    --> need to stitch points as:
    Regular: current_point,other_1,other_2
    Neighbor: current,current-1,other
    """


    #print(f"index of bigger facets = {bigger}\nindex of smaller facets = {smaller}",)

    #calculates the number of vertices will be stitched to each vertices on smaller side
    dividend = int(lengths_of_boundaries[bigger]/lengths_of_boundaries[smaller])
    remainder = lengths_of_boundaries[bigger] - int(lengths_of_boundaries[bigger]/lengths_of_boundaries[smaller])*lengths_of_boundaries[smaller]

    print(f"dividend = {dividend}, remainder = {remainder}")

    #loop that adds the new faces
    print("About to add faces")
    start_time = time.time()
    new_faces = []
    current_bigger = 0

    for i,current_smaller in enumerate(edge_order_list[smaller]):
        #print("current_smaller =" + str(current_smaller))
        #print("current_bigger=" + str(edge_order_list[bigger][current_bigger]))

        #connecting to the neighbor on the shorter side
        """
        if i == 0:

            new_faces.append([current_smaller,edge_order_list[smaller][-1],edge_order_list[bigger][current_bigger]])
        else:
            new_faces.append([current_smaller,edge_order_list[smaller][i-1],edge_order_list[bigger][current_bigger]])
        """


        if starter_face == "main":
            new_faces.append([current_smaller,edge_order_list[bigger][current_bigger],edge_order_list[smaller][i-1]])
        else:
            new_faces.append([current_smaller,edge_order_list[smaller][i-1],edge_order_list[bigger][current_bigger]])


        for j in range(0,dividend + int(i<remainder)):
            if current_bigger > len(edge_order_list[bigger]):
                raise Exception("Somehow rapped around too much")

            if current_bigger >= len(edge_order_list[bigger])-1:
                next_bigger = 0
            else:
                next_bigger = current_bigger+1

            if starter_face == "main":
                new_faces.append([edge_order_list[bigger][next_bigger],
                                  edge_order_list[bigger][current_bigger],
                                  current_smaller,
                                ])
            else:
                new_faces.append([
                                current_smaller,
                                  edge_order_list[bigger][current_bigger],
                                  edge_order_list[bigger][next_bigger],

                                ])

            current_bigger += 1





    #print("new_faces = " + str(new_faces))
    print(f"Finished adding faces: {time.time() - start_time}")


    print("Starting creating stitch mesh")
    start_time = time.time()
    stitch_mesh = trimesh.Trimesh()

    stitch_mesh.vertices = new_mesh.vertices
    stitch_mesh.faces = np.vstack([new_mesh.faces, new_faces])
    print(f"Finished creating stitch mesh: {time.time() - start_time}")




    if delete_facets == True:
        #now take away the original facet faces:
        total_faces = np.linspace(0,len(stitch_mesh.faces)-1,len(stitch_mesh.faces)).astype("int")
        facet_faces = np.hstack([facet_1 ,facet_2])
        faces_to_keep = set(total_faces).difference(set(facet_faces))
        faces_to_keep

        stitch_mesh = stitch_mesh.submesh([list(faces_to_keep)])[0]

    if fix_normals == True:
        trimesh.repair.fix_inversion(stitch_mesh)
        trimesh.repair.fix_winding(stitch_mesh)
        trimesh.repair.fix_normals(stitch_mesh)

    #print("Finished stitching")

    if return_added_mesh == True:
        added_mesh = trimesh.Trimesh()
        added_mesh.vertices = new_mesh.vertices
        added_mesh.faces = new_faces
        trimesh.repair.fix_inversion(added_mesh)
        trimesh.repair.fix_winding(added_mesh)
        trimesh.repair.fix_normals(added_mesh)

        return stitch_mesh,added_mesh

    else:
        return stitch_mesh

In [97]:
# #find the distance between the normals:
# big_Main_normal = [0.01792991, 0.02972343, 0.99939734]
# big_Child_normal = [ 2.25293788e-02,  8.23087907e-04, -9.99745843e-01]
# Main_normal = [-1.07733293e-05,  9.99999840e-01,  5.66120655e-04]
# Child_normal = [-0.00323315, -0.9996487 , -0.02630649]

# np.dot(big_Main_normal,Main_normal)
# np.dot(big_Child_normal,Child_normal)

# np.dot(big_Child_normal,Main_normal)


In [98]:
main_mesh = trimesh.load_mesh("./neurons/neuron_648518346349482676.off")





In [99]:
#create the output file
##write the OFF file for the neuron
import pathlib
def write_Whole_Neuron_Off_file(neuron_ID,
                                vertices=[], 
                                triangles=[],
                                folder="pymesh_neurons"):
    #primary_key = dict(segmentation=1, segment_id=segment_id, decimation_ratio=0.35)
    #vertices, triangles = (mesh_Table_35 & primary_key).fetch1('vertices', 'triangles')
    
    num_vertices = (len(vertices))
    num_faces = len(triangles)
    
    #get the current file location
    file_loc = pathlib.Path.cwd() / folder
    filename = "neuron_" + str(neuron_ID)
    path_and_filename = file_loc / filename
    
    #print(file_loc)
    #print(path_and_filename)
    
    #open the file and start writing to it    
    f = open(str(path_and_filename) + ".off", "w")
    f.write("OFF\n")
    f.write(str(num_vertices) + " " + str(num_faces) + " 0\n" )
    
    
    #iterate through and write all of the vertices in the file
    for verts in vertices:
        f.write(str(verts[0]) + " " + str(verts[1]) + " " + str(verts[2])+"\n")
    
    #print("Done writing verts")
        
    for faces in triangles:
        f.write("3 " + str(faces[0]) + " " + str(faces[1]) + " " + str(faces[2])+"\n")
    
    print("Done writing OFF file")
    #f.write("end")
    
    return str(path_and_filename),str(filename),str(file_loc)

In [100]:
#write_Whole_Neuron_Off_file(648518346349482676,main_mesh.vertices,main_mesh.faces,"neurons")

In [101]:

facet_2 = [1194282 ,1194901, 1194900 ,1194271, 1194219, 1195037, 1193615 ,1193587 ,1193582,1193572 ,1194930 ,1194258, 1193937, 1194864]
    
facet_1 = [168738, 169294, 169253, 168913, 169292, 170133, 170134, 168912]
    
stitch_mesh = stitch_mesh_piece_vp4(main_mesh,facet_1,facet_2)
                          

statistcs on the entire mesh
len(new_mesh.faces) = 1260719
len(new_mesh.vertices) = 634662
facet_1 = [168738, 169294, 169253, 168913, 169292, 170133, 170134, 168912]
facet_2 = [1194282, 1194901, 1194900, 1194271, 1194219, 1195037, 1193615, 1193587, 1193582, 1193572, 1194930, 1194258, 1193937, 1194864]
Main normal = [0.01792991 0.02972343 0.99939734]
Child normal = [ 2.25293788e-02  8.23087907e-04 -9.99745843e-01]
opposite direction normals
edges_facet = [array([[ 96159, 335732],
       [ 96159, 309399],
       [309399, 335732],
       [  6386, 490253],
       [428105, 490253],
       [  6386, 428105],
       [ 96159, 423272],
       [423272, 428105],
       [ 96159, 428105],
       [309399, 428105],
       [312497, 428105],
       [309399, 312497],
       [ 96159, 309399],
       [ 96159, 428105],
       [309399, 428105],
       [312497, 490253],
       [490253, 577835],
       [312497, 577835],
       [312497, 428105],
       [428105, 490253],
       [312497, 490253],
       [309399, 

In [102]:
from print_trimesh import print_trimesh
print_trimesh(stitch_mesh,"./neurons/648518346349482676_stitch_test_result_2.off")

In [56]:
test = trimesh.load_mesh("./neurons/648518346349482676_stitch_test_result.off")
test.show()

KeyboardInterrupt: 

In [90]:
example_loop = [4,6,2,8,10,13,12,3]


#new_loop = [example_loop[0]]
new_loop = [example_loop[len(example_loop) - x -1] for x in range(0,len(example_loop)-1)]
new_loop.insert(0,example_loop[0])
#total_loop = new_loop + addon
x = new_loop.copy()
print(x)

[4, 3, 12, 13, 10, 8, 2, 6]


In [92]:
new_loop[0] = 5
new_loop

[5, 3, 12, 13, 10, 8, 2, 6]

In [93]:
x

[4, 3, 12, 13, 10, 8, 2, 6]